In [ ]:
from flask import Flask, render_template_string, request, Response
import pyodbc
import pandas as pd
import io

app = Flask(__name__)

html_template = '''
<!DOCTYPE html>
<html>
<head>
    <title>Records</title>
    <style>
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background-color: #f7f7f7;
            margin: 20px;
        }

        h1 {
            color: #FF7518;
            text-align: center;
            margin-bottom: 20px;
        }

        form {
            padding: 15px;
            background-color: #ffffff;
            border: 1px solid #cccccc;
            border-radius: 5px;
            max-width: 400px;
            margin: 0 auto;
            box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
        }

        label {
            display: block;
            margin-bottom: 5px;
            color: #333333;
        }

        input[type="text"] {
            width: 100%;
            padding: 10px;
            margin-bottom: 15px;
            border: 1px solid #cccccc;
            border-radius: 4px;
        }

        #var1, #var2 {
            width: 40%;
            margin-right: 10%;
        }

        button[type="submit"] {
            background-color: #000000;
            color: #ffffff;
            padding: 12px 18px;
            border: none;
            border-radius: 4px;
            cursor: pointer;
            width: 100%;
            transition: background-color 0.3s ease;
        }

        button[type="submit"]:hover {
            background-color: #204d74;
        }

        p {
            color: #333333;
            margin-top: 20px;
        }

        table {
            width: 100%;
            border-collapse: collapse;
            margin-top: 30px;
        }

        th {
            background-color: #000000;
            color: #ffffff;
            padding: 12px;
            text-align: left;
        }

        td {
            padding: 10px;
            border-bottom: 1px solid #cccccc;
        }

        tr:hover {
            background-color: #f2f2f2;
        }
    </style>
</head>
<body>
    <h1>Records</h1>
    <form method="POST" action="/">
        <div id="#">
            <label for="#"> ## :</label>
            <input type="text" id="#" name="#">
        </div>
        
        <div id=" # ">
            <label for="#"> ## :</label>
            <input type="text" id=" # " name="#">
        </div>
        
        <div class="clear"></div>
        <br>
        <button type="submit">Submit</button>
    </form>
    {% if count is not none %}
    <p>Count: {{ count }}</p>
    {% if records %}
    <form method="post" action="/download">
        <input type="hidden" name="#" value="{{ ## }}">
        <input type="hidden" name="#" value="{{ ## }}">
        <button type="submit">Download Records (Excel)</button>
    </form>
    <br>
    <h2>Sample Records</h2>
    <table border="1">
        <tr>
            {% for column in column_names %}
            <th>{{ column }}</th>
            {% endfor %}
        </tr>
        {% for row in records[:10] %}  
        <tr>
            {% for value in row %}
            <td>{{ value }}</td>
            {% endfor %}
        </tr>
        {% endfor %}
    </table>
    {% endif %}
    {% endif %}
</body>
</html>
'''


@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        var1 = request.form.get('var1')
        var2 = request.form.get('var2')

        server =   'your_server_name'
        database = 'your_database_name'
        username = 'your_user_id'
        password = 'your_password'
        connection = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database + ';UID=' + username + ';PWD=' + password)
        cursor = connection.cursor()

        if var1 and var2:
            query = "SELECT top 10 * FROM your_table _name with(nolock) WHERE your_condition"
            params = (var1, var2)
            cursor.execute(query, params)
            records = cursor.fetchall()
            count = len(records)
        else:
            query = "SELECT top 10 * FROM your_table_name with(nolock)"
            cursor.execute(query)
            records = cursor.fetchall()
            count = len(records)

       
        column_names = [column[0] for column in cursor.description]

        cursor.close()
        connection.close()

        return render_template_string(html_template, count=count, var1=var1, var2=var2, records=records, column_names=column_names)

    return render_template_string(html_template, count=None)

@app.route('/download', methods=['POST'])
def download():
    var1 = request.form.get('var1')
    var2 = request.form.get('var2')

    server =   'your_server_name'
    database = 'your_database_name'
    username = 'your_user_id'
    password = 'your_password'
    connection = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database + ';UID=' + username + ';PWD=' + password)
    cursor = connection.cursor()

    query = "SELECT top 10  * FROM your_table_name with(nolock) WHERE your_condition with(nolock)"
    params = (var1, var2)
    cursor.execute(query, params)
    records = cursor.fetchall()

   
    column_names = [column[0] for column in cursor.description]

    cursor.close()
    connection.close()

    if records:
        df = pd.DataFrame.from_records(records, columns=column_names)

       
        excel_output = io.BytesIO()
        with pd.ExcelWriter(excel_output, engine='xlsxwriter') as writer:
            df.to_excel(writer, index=False, sheet_name='your_sheet_name')

      
        excel_output.seek(0)
        response = Response(excel_output.getvalue(), content_type='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
        response.headers['Content-Disposition'] = 'attachment; filename= your_file_name.xlsx'

        return response

    return "No records found."

if __name__ == '__main__':
    app.run(debug=True)
